In [57]:
# #!/usr/bin/env python3
# #
# # Team members' IDs:
# # 182a3da8-8a9e-11ec-986f-f39926f24a9c  (Jan Zubáč)
# # 7797f596-9326-11ec-986f-f39926f24a9c
# # 449dba85-9adb-11ec-986f-f39926f24a9c
# #
# #
# import argparse
# import datetime
# import os
# import re
# os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

# import numpy as np
# import tensorflow as tf

# from modelnet import ModelNet

# # TODO: Define reasonable defaults and optionally more parameters.
# # Also, you can set the number of the threads 0 to use all your CPU cores.
# # parser = argparse.ArgumentParser()
# # parser.add_argument("--batch_size", default=64, type=int, help="Batch size.")
# # parser.add_argument("--epochs", default=1, type=int, help="Number of epochs.")
# # parser.add_argument("--modelnet", default=20, type=int, help="ModelNet dimension.")
# # parser.add_argument("--seed", default=42, type=int, help="Random seed.")
# # parser.add_argument("--threads", default=1, type=int, help="Maximum number of threads to use.")

# # parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
# # parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
# # parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
# # parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
# # parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
# # parser.add_argument("--dropout", default=0.3, type=float, help="Dropout regularization.")
# # parser.add_argument("--hidden_layers", default=[100,100], nargs="*", type=int, help="Hidden layer sizes.")
# # parser.add_argument("--l2", default=0.0, type=float, help="L2 regularization.")

# # parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")


# def main(args: argparse.Namespace) -> None:
#     # Fix random seeds and threads
#     tf.keras.utils.set_random_seed(args.seed)
#     tf.config.threading.set_inter_op_parallelism_threads(args.threads)
#     tf.config.threading.set_intra_op_parallelism_threads(args.threads)

#     # Create logdir name
#     args.logdir = os.path.join("logs", "{}-{}-{}".format(
#         os.path.basename(globals().get("__file__", "notebook")),
#         datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
#         ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
#     ))

#     # Load the data
#     modelnet = ModelNet(args.modelnet)

#     inputs = tf.keras.layers.Input(shape=[modelnet.D, modelnet.H, modelnet.W, modelnet.C], dtype=tf.float32)

#     ## conv3D layers
#     out = inputs
#     for i, filter_size in enumerate(args.filter_sizes):
#             conv3D =  tf.keras.layers.Conv3D(filters=filter_size,
#                                         kernel_size=3,
#                                         strides=(2, 2, 2),
#                                         padding='same',
#                                         data_format="channels_last",
#                                         kernel_regularizer=tf.keras.regularizers.L2(l2=args.l2))
#             out = conv3D (out)

#             # if i != len(args.filter_sizes)-1:
#             #   out = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2)) (out)
#             out = tf.keras.layers.BatchNormalization()(out)
#             out = tf.keras.layers.Dropout(args.dropout)(out)
#             # print("out", out.shape)                                

#     out = tf.keras.layers.Flatten()(out)

#     ## dense layers
#     hidden = out
#     for hidden_layer_size in args.hidden_layers:
#         hidden =  tf.keras.layers.Dense(hidden_layer_size, activation=tf.nn.relu) (hidden)
#         hidden = tf.keras.layers.Dropout(args.dropout) (hidden)

#     outputs = tf.keras.layers.Dense(len(modelnet.LABELS), activation=tf.nn.softmax)(hidden)
    
#     # choosing lr
#     learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(  args.learning_rate, args.decay_steps, 
#                                                                     end_learning_rate=args.learning_rate_final, power=1.0,
#                                                                     cycle=False, name='linear_lr_decay')

#     # choosing optimizer
#     if args.optimizer=='SGD':
#         if args.momentum != None:
#             momentum = args.momentum
#         else:
#             momentum = 0.0
#         optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=False, name='SGD')

#     if args.optimizer=='Adam':
#         optimizer =  tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')



#     # TODO: Create the model and train it
#     model = tf.keras.Model(inputs=inputs, outputs=outputs)
#     model.compile(
#         optimizer=optimizer,
#         loss=tf.losses.SparseCategoricalCrossentropy(),
#         metrics=[tf.metrics.SparseCategoricalAccuracy("accuracy")],
#         )
    

#     # # train, dev = modelnet.train.data, modelnet.dev.data


#     #     # # Create input data pipeline.
#     def create_dataset(name):

#         dataset = getattr(modelnet, name).dataset
#         dataset = dataset.map(lambda example: (example["voxels"], example["labels"]))
#         dataset = dataset.shuffle(len(dataset), seed=args.seed) if name == "train" else dataset
#         dataset = dataset.batch(args.batch_size)
#         return dataset

#     train, dev, test = create_dataset("train"), create_dataset("dev"), create_dataset("test")


#     tb_callback = tf.keras.callbacks.TensorBoard(args.logdir, update_freq=100, profile_batch=0)
#     logs = model.fit(train, epochs=args.epochs, validation_data=dev)

#     # Generate test set annotations, but in `args.logdir` to allow parallel execution.
#     os.makedirs(args.logdir, exist_ok=True)
#     with open(os.path.join(args.logdir, "3d_recognition.txt"), "w", encoding="utf-8") as predictions_file:
#         # TODO: Predict the probabilities on the test set
#         test_probabilities = model.predict(test)

#         for probs in test_probabilities:
#             print(np.argmax(probs), file=predictions_file)


# # if __name__ == "__main__":
# #     args = parser.parse_args([] if "__file__" not in globals() else None)
# #     main(args)


In [45]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.6, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[10], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.6, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[10], l2=0.05, learning_rate=0.001, learning_rate_final=0.0005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 2s 43ms/step - loss: 16.3492 - accuracy: 0.1143 - val_loss: 14.7374 - val_accuracy: 0.1465
Epoch 2/50
15/15 [==============================] - 0s 23ms/step - loss: 15.9165 - accuracy: 0.1224 - val_loss: 14.5080 - val_accuracy: 0.1905
Epoch 3/50
15/15 [==============================] - 0s 23ms/step - loss: 15.5600 - accuracy: 0.1237 - val_loss: 14.2741 - val_accuracy: 0.2051
Epoch 4/50
15/15 [==============================] - 0s 24ms/step - loss: 15.0555 - accuracy: 0.1417 - val_loss: 14.0386 - val_accuracy: 0.2527
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 14.5905 - accuracy: 0.1557 - val_loss: 13.7936 - val_accuracy: 0.2747
Epoch 6/50
15/15 [====================

In [46]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.6, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.6, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.001, learning_rate_final=0.0005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 43ms/step - loss: 16.3384 - accuracy: 0.1173 - val_loss: 14.7234 - val_accuracy: 0.2674
Epoch 2/50
15/15 [==============================] - 0s 24ms/step - loss: 15.9105 - accuracy: 0.1326 - val_loss: 14.4470 - val_accuracy: 0.4103
Epoch 3/50
15/15 [==============================] - 0s 24ms/step - loss: 15.2651 - accuracy: 0.1815 - val_loss: 14.1001 - val_accuracy: 0.5421
Epoch 4/50
15/15 [==============================] - 0s 24ms/step - loss: 14.3744 - accuracy: 0.3098 - val_loss: 13.6129 - val_accuracy: 0.5971
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 13.5702 - accuracy: 0.4494 - val_loss: 13.0484 - val_accuracy: 0.6227
Epoch 6/50
15/15 [======================

In [47]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.6, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.6, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.6, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.001, learning_rate_final=0.0005, modelnet=32, momentum=0.6, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 43ms/step - loss: 16.3939 - accuracy: 0.1132 - val_loss: 14.8105 - val_accuracy: 0.1538
Epoch 2/50
15/15 [==============================] - 0s 24ms/step - loss: 16.2586 - accuracy: 0.1151 - val_loss: 14.7044 - val_accuracy: 0.2894
Epoch 3/50
15/15 [==============================] - 0s 24ms/step - loss: 16.0511 - accuracy: 0.1232 - val_loss: 14.5933 - val_accuracy: 0.3443
Epoch 4/50
15/15 [==============================] - 0s 24ms/step - loss: 15.7563 - accuracy: 0.1304 - val_loss: 14.4717 - val_accuracy: 0.4359
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 15.4887 - accuracy: 0.1708 - val_loss: 14.3319 - val_accuracy: 0.4872
Epoch 6/50
15/15 [=======================

In [48]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.4, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.4, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.001, learning_rate_final=0.0005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 45ms/step - loss: 15.7596 - accuracy: 0.1337 - val_loss: 14.6957 - val_accuracy: 0.3736
Epoch 2/50
15/15 [==============================] - 0s 24ms/step - loss: 14.5120 - accuracy: 0.3521 - val_loss: 14.2812 - val_accuracy: 0.6007
Epoch 3/50
15/15 [==============================] - 0s 24ms/step - loss: 13.4367 - accuracy: 0.6054 - val_loss: 13.7404 - val_accuracy: 0.6520
Epoch 4/50
15/15 [==============================] - 0s 24ms/step - loss: 12.9000 - accuracy: 0.7033 - val_loss: 13.2551 - val_accuracy: 0.6886
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 12.5254 - accuracy: 0.7364 - val_loss: 12.8517 - val_accuracy: 0.7216
Epoch 6/50
15/15 [======================

In [49]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.5, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.5, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.001, learning_rate_final=0.0005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 42ms/step - loss: 16.0402 - accuracy: 0.1194 - val_loss: 14.7109 - val_accuracy: 0.3407
Epoch 2/50
15/15 [==============================] - 0s 24ms/step - loss: 15.2726 - accuracy: 0.1977 - val_loss: 14.3749 - val_accuracy: 0.5311
Epoch 3/50
15/15 [==============================] - 0s 24ms/step - loss: 14.2081 - accuracy: 0.3954 - val_loss: 13.8684 - val_accuracy: 0.6044
Epoch 4/50
15/15 [==============================] - 0s 24ms/step - loss: 13.3236 - accuracy: 0.5845 - val_loss: 13.3002 - val_accuracy: 0.6557
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 12.8025 - accuracy: 0.6576 - val_loss: 12.8424 - val_accuracy: 0.6886
Epoch 6/50
15/15 [======================

In [50]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.3, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.3, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.001, learning_rate_final=0.0005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 43ms/step - loss: 15.4233 - accuracy: 0.1724 - val_loss: 14.6783 - val_accuracy: 0.4505
Epoch 2/50
15/15 [==============================] - 0s 23ms/step - loss: 13.9307 - accuracy: 0.5169 - val_loss: 14.2193 - val_accuracy: 0.6337
Epoch 3/50
15/15 [==============================] - 0s 23ms/step - loss: 13.0948 - accuracy: 0.7058 - val_loss: 13.7374 - val_accuracy: 0.6996
Epoch 4/50
15/15 [==============================] - 0s 24ms/step - loss: 12.6658 - accuracy: 0.7595 - val_loss: 13.3120 - val_accuracy: 0.7326
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 12.3317 - accuracy: 0.7929 - val_loss: 12.9297 - val_accuracy: 0.7619
Epoch 6/50
15/15 [======================

In [51]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.4, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.4, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.01, learning_rate_final=0.005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 43ms/step - loss: 14.0200 - accuracy: 0.4640 - val_loss: 12.5002 - val_accuracy: 0.5934
Epoch 2/50
15/15 [==============================] - 0s 24ms/step - loss: 11.0588 - accuracy: 0.7959 - val_loss: 10.2632 - val_accuracy: 0.6960
Epoch 3/50
15/15 [==============================] - 0s 23ms/step - loss: 9.0798 - accuracy: 0.8491 - val_loss: 8.6162 - val_accuracy: 0.7875
Epoch 4/50
15/15 [==============================] - 0s 23ms/step - loss: 7.4831 - accuracy: 0.8693 - val_loss: 7.3777 - val_accuracy: 0.7949
Epoch 5/50
15/15 [==============================] - 0s 23ms/step - loss: 6.1930 - accuracy: 0.8685 - val_loss: 6.4489 - val_accuracy: 0.7692
Epoch 6/50
15/15 [==============================

In [53]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=50, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.0001, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.0001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.4, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.4, epochs=50, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.0001, learning_rate_final=0.0001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/50
15/15 [==============================] - 1s 43ms/step - loss: 16.0712 - accuracy: 0.0974 - val_loss: 14.8842 - val_accuracy: 0.0952
Epoch 2/50
15/15 [==============================] - 0s 24ms/step - loss: 15.8985 - accuracy: 0.1146 - val_loss: 14.8507 - val_accuracy: 0.1722
Epoch 3/50
15/15 [==============================] - 0s 23ms/step - loss: 15.7668 - accuracy: 0.1237 - val_loss: 14.8135 - val_accuracy: 0.2491
Epoch 4/50
15/15 [==============================] - 0s 23ms/step - loss: 15.6648 - accuracy: 0.1447 - val_loss: 14.7718 - val_accuracy: 0.3077
Epoch 5/50
15/15 [==============================] - 0s 24ms/step - loss: 15.4482 - accuracy: 0.1735 - val_loss: 14.7241 - val_accuracy: 0.3590
Epoch 6/50
15/15 [=====================

In [54]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=500, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.4, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=1000, dropout=0.4, epochs=500, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.05, learning_rate=0.01, learning_rate_final=0.005, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/500
15/15 [==============================] - 1s 44ms/step - loss: 14.0200 - accuracy: 0.4640 - val_loss: 12.5002 - val_accuracy: 0.5934
Epoch 2/500
15/15 [==============================] - 0s 23ms/step - loss: 11.0588 - accuracy: 0.7959 - val_loss: 10.2632 - val_accuracy: 0.6960
Epoch 3/500
15/15 [==============================] - 0s 24ms/step - loss: 9.0798 - accuracy: 0.8491 - val_loss: 8.6162 - val_accuracy: 0.7875
Epoch 4/500
15/15 [==============================] - 0s 23ms/step - loss: 7.4831 - accuracy: 0.8693 - val_loss: 7.3777 - val_accuracy: 0.7949
Epoch 5/500
15/15 [==============================] - 0s 24ms/step - loss: 6.1930 - accuracy: 0.8685 - val_loss: 6.4489 - val_accuracy: 0.7692
Epoch 6/500
15/15 [=======================

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=500, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.005, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.4, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.05, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

In [54]:
batch_size=256, 
decay_steps=100, 
dropout=0.6, 
epochs=150, 
filter_sizes=[4, 16, 48, 128, 192], 
hidden_layers=[], 
l2=0.15, learning_rate=0.01,
 learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='15SGD', seed=42, threads=0

In [56]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=150, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.55, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.15, type=float, help="L2 regularization.")


parser.add_argument("--filter_sizes", default=[2, 16, 48, 128, 192] ,  nargs="*", type=int, help="Filters for deconv-")
# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=100, dropout=0.55, epochs=150, filter_sizes=[2, 16, 48, 128, 192], hidden_layers=[], l2=0.15, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/150
15/15 [==============================] - 1s 43ms/step - loss: 35.9707 - accuracy: 0.3123 - val_loss: 27.7213 - val_accuracy: 0.6044
Epoch 2/150
15/15 [==============================] - 0s 23ms/step - loss: 22.0391 - accuracy: 0.7396 - val_loss: 16.9610 - val_accuracy: 0.6777
Epoch 3/150
15/15 [==============================] - 0s 23ms/step - loss: 13.5657 - accuracy: 0.8031 - val_loss: 11.4947 - val_accuracy: 0.7656
Epoch 4/150
15/15 [==============================] - 0s 23ms/step - loss: 8.9919 - accuracy: 0.8378 - val_loss: 8.7616 - val_accuracy: 0.4872
Epoch 5/150
15/15 [==============================] - 0s 24ms/step - loss: 6.6069 - accuracy: 0.8375 - val_loss: 7.2578 - val_accuracy: 0.2344
Epoch 6/150
15/15 [=====================

In [1]:
#!/usr/bin/env python3
#
# Team members' IDs:
# 182a3da8-8a9e-11ec-986f-f39926f24a9c  (Jan Zubáč)
# 7797f596-9326-11ec-986f-f39926f24a9c
# 449dba85-9adb-11ec-986f-f39926f24a9c
#
#
import argparse
import datetime
import os
import re
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import numpy as np
import tensorflow as tf

from modelnet import ModelNet

# TODO: Define reasonable defaults and optionally more parameters.
# Also, you can set the number of the threads 0 to use all your CPU cores.
# parser = argparse.ArgumentParser()
# parser.add_argument("--batch_size", default=64, type=int, help="Batch size.")
# parser.add_argument("--epochs", default=1, type=int, help="Number of epochs.")
# parser.add_argument("--modelnet", default=20, type=int, help="ModelNet dimension.")
# parser.add_argument("--seed", default=42, type=int, help="Random seed.")
# parser.add_argument("--threads", default=1, type=int, help="Maximum number of threads to use.")

# parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
# parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
# parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
# parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
# parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
# parser.add_argument("--dropout", default=0.3, type=float, help="Dropout regularization.")
# parser.add_argument("--hidden_layers", default=[100,100], nargs="*", type=int, help="Hidden layer sizes.")
# parser.add_argument("--l2", default=0.0, type=float, help="L2 regularization.")

# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")


def main(args: argparse.Namespace) -> None:
    # Fix random seeds and threads
    tf.keras.utils.set_random_seed(args.seed)
    tf.config.threading.set_inter_op_parallelism_threads(args.threads)
    tf.config.threading.set_intra_op_parallelism_threads(args.threads)

    # Create logdir name
    args.logdir = os.path.join("logs", "{}-{}-{}".format(
        os.path.basename(globals().get("__file__", "notebook")),
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    ))

    # Load the data
    modelnet = ModelNet(args.modelnet)

    inputs = tf.keras.layers.Input(shape=[modelnet.D, modelnet.H, modelnet.W, modelnet.C], dtype=tf.float32)

    ## conv3D layers
    out = inputs
    for i, filter_size in enumerate(args.filter_sizes):
            conv3D =  tf.keras.layers.Conv3D(filters=filter_size,
                                        kernel_size=3,
                                        strides=(1, 1, 1),
                                        padding='same',
                                        data_format="channels_last",
                                        kernel_regularizer=tf.keras.regularizers.L2(l2=args.l2))
            out = conv3D (out)

            # if i != len(args.filter_sizes)-1:
            out = tf.keras.layers.MaxPool3D(pool_size=(3, 3, 3)) (out)
            out = tf.keras.layers.BatchNormalization()(out)
            out = tf.keras.layers.BatchNormalization()(out)
            # out = tf.keras.layers.Dropout(args.dropout)(out)
            # print("out", out.shape)                                

    out = tf.keras.layers.Flatten()(out)

    ## dense layers
    hidden = out
    for hidden_layer_size in args.hidden_layers:
        hidden =  tf.keras.layers.Dense(hidden_layer_size, activation=tf.nn.relu) (hidden)
        hidden = tf.keras.layers.Dropout(args.dropout) (hidden)

    outputs = tf.keras.layers.Dense(len(modelnet.LABELS), activation=tf.nn.softmax)(hidden)

(modelnet.LABELS), activation="softmax")(x)
    
    # choosing lr
    learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(  args.learning_rate, args.decay_steps, 
                                                                    end_learning_rate=args.learning_rate_final, power=1.0,
                                                                    cycle=False, name='linear_lr_decay')

    # choosing optimizer
    if args.optimizer=='SGD':
        if args.momentum != None:
            momentum = args.momentum
        else:
            momentum = 0.0
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=False, name='SGD')

    if args.optimizer=='Adam':
        optimizer =  tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')



    # TODO: Create the model and train it
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=optimizer,
        loss=tf.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.metrics.SparseCategoricalAccuracy("accuracy")],
        )
    

    # # train, dev = modelnet.train.data, modelnet.dev.data


    #     # # Create input data pipeline.
    def create_dataset(name):

        dataset = getattr(modelnet, name).dataset
        dataset = dataset.map(lambda example: (example["voxels"], example["labels"]))
        dataset = dataset.shuffle(len(dataset), seed=args.seed) if name == "train" else dataset
        dataset = dataset.batch(args.batch_size)
        return dataset

    train, dev, test = create_dataset("train"), create_dataset("dev"), create_dataset("test")


    tb_callback = tf.keras.callbacks.TensorBoard(args.logdir, update_freq=100, profile_batch=0)
    logs = model.fit(train, epochs=args.epochs, validation_data=dev)

    # Generate test set annotations, but in `args.logdir` to allow parallel execution.
    os.makedirs(args.logdir, exist_ok=True)
    with open(os.path.join(args.logdir, "3d_recognition.txt"), "w", encoding="utf-8") as predictions_file:
        # TODO: Predict the probabilities on the test set
        test_probabilities = model.predict(test)

        for probs in test_probabilities:
            print(np.argmax(probs), file=predictions_file)


# if __name__ == "__main__":
#     args = parser.parse_args([] if "__file__" not in globals() else None)
#     main(args)


In [113]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.6, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[16,32,64] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.6, epochs=100, filter_sizes=[16, 32, 64], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 2s 98ms/step - loss: 2.6870 - accuracy: 0.3004 - val_loss: 2.8819 - val_accuracy: 0.2821
Epoch 2/100
15/15 [==============================] - 1s 82ms/step - loss: 1.7167 - accuracy: 0.7028 - val_loss: 2.7384 - val_accuracy: 0.3370
Epoch 3/100
15/15 [==============================] - 1s 82ms/step - loss: 1.2883 - accuracy: 0.7972 - val_loss: 2.7910 - val_accuracy: 0.2088
Epoch 4/100
15/15 [==============================] - 1s 82ms/step - loss: 1.0823 - accuracy: 0.8564 - val_loss: 2.9266 - val_accuracy: 0.2088
Epoch 5/100
15/15 [==============================] - 1s 82ms/step - loss: 0.9713 - accuracy: 0.8808 - val_loss: 3.1259 - val_accuracy: 0.1978
Epoch 6/100
15/15 [===========================

In [114]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.5, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[16,32,64] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.5, epochs=100, filter_sizes=[16, 32, 64], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 2s 96ms/step - loss: 2.6188 - accuracy: 0.3445 - val_loss: 2.8778 - val_accuracy: 0.3150
Epoch 2/100
15/15 [==============================] - 1s 80ms/step - loss: 1.6351 - accuracy: 0.7259 - val_loss: 2.7405 - val_accuracy: 0.3114
Epoch 3/100
15/15 [==============================] - 1s 80ms/step - loss: 1.2314 - accuracy: 0.8147 - val_loss: 2.7829 - val_accuracy: 0.2088
Epoch 4/100
15/15 [==============================] - 1s 80ms/step - loss: 1.0432 - accuracy: 0.8644 - val_loss: 2.8965 - val_accuracy: 0.2051
Epoch 5/100
15/15 [==============================] - 1s 80ms/step - loss: 0.9299 - accuracy: 0.8919 - val_loss: 3.0603 - val_accuracy: 0.1905
Epoch 6/100
15/15 [===========================

In [115]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 4s 184ms/step - loss: 3.1080 - accuracy: 0.4088 - val_loss: 3.4446 - val_accuracy: 0.4615
Epoch 2/100
15/15 [==============================] - 2s 143ms/step - loss: 2.0726 - accuracy: 0.7579 - val_loss: 3.3553 - val_accuracy: 0.3150
Epoch 3/100
15/15 [==============================] - 2s 143ms/step - loss: 1.7596 - accuracy: 0.8260 - val_loss: 3.4090 - val_accuracy: 0.2088
Epoch 4/100
15/15 [==============================] - 2s 143ms/step - loss: 1.5816 - accuracy: 0.8712 - val_loss: 3.5510 - val_accuracy: 0.1868
Epoch 5/100
15/15 [==============================] - 2s 143ms/step - loss: 1.4701 - accuracy: 0.8916 - val_loss: 3.6552 - val_accuracy: 0.1209
Epoch 6/100
15/15 [====================

In [116]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[64,128,256] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=100, filter_sizes=[64, 128, 256], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 8s 395ms/step - loss: 4.2822 - accuracy: 0.4793 - val_loss: 4.6711 - val_accuracy: 0.3260
Epoch 2/100
15/15 [==============================] - 5s 310ms/step - loss: 3.1851 - accuracy: 0.7821 - val_loss: 4.6454 - val_accuracy: 0.2674
Epoch 3/100
15/15 [==============================] - 5s 310ms/step - loss: 2.8821 - accuracy: 0.8556 - val_loss: 4.7359 - val_accuracy: 0.2161
Epoch 4/100
15/15 [==============================] - 5s 310ms/step - loss: 2.6725 - accuracy: 0.8997 - val_loss: 4.9181 - val_accuracy: 0.1905
Epoch 5/100
15/15 [==============================] - 5s 311ms/step - loss: 2.5379 - accuracy: 0.9094 - val_loss: 5.1076 - val_accuracy: 0.1319
Epoch 6/100
15/15 [===================

In [123]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=250, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66,type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=250, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/250
15/15 [==============================] - 3s 157ms/step - loss: 3.1072 - accuracy: 0.4104 - val_loss: 3.4492 - val_accuracy: 0.4689
Epoch 2/250
15/15 [==============================] - 2s 144ms/step - loss: 2.0706 - accuracy: 0.7587 - val_loss: 3.3594 - val_accuracy: 0.3150
Epoch 3/250
15/15 [==============================] - 2s 144ms/step - loss: 1.7594 - accuracy: 0.8222 - val_loss: 3.4205 - val_accuracy: 0.2125
Epoch 4/250
15/15 [==============================] - 2s 143ms/step - loss: 1.5783 - accuracy: 0.8696 - val_loss: 3.5648 - val_accuracy: 0.2088
Epoch 5/250
15/15 [==============================] - 2s 144ms/step - loss: 1.4717 - accuracy: 0.8938 - val_loss: 3.6688 - val_accuracy: 0.1538
Epoch 6/250
15/15 [====================

In [125]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66,type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,16,8] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=100, filter_sizes=[32, 16, 8], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 3s 145ms/step - loss: 2.5561 - accuracy: 0.2012 - val_loss: 2.6146 - val_accuracy: 0.1099
Epoch 2/100
15/15 [==============================] - 2s 126ms/step - loss: 2.1728 - accuracy: 0.4817 - val_loss: 2.5673 - val_accuracy: 0.2857
Epoch 3/100
15/15 [==============================] - 2s 125ms/step - loss: 1.7109 - accuracy: 0.5976 - val_loss: 2.5285 - val_accuracy: 0.2271
Epoch 4/100
15/15 [==============================] - 2s 126ms/step - loss: 1.4236 - accuracy: 0.6366 - val_loss: 2.5304 - val_accuracy: 0.2088
Epoch 5/100
15/15 [==============================] - 2s 125ms/step - loss: 1.2442 - accuracy: 0.6872 - val_loss: 2.5903 - val_accuracy: 0.1502
Epoch 6/100
15/15 [======================

In [127]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
# parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")
parser.add_argument("--filter_sizes", default=[64,32,16] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=100, filter_sizes=[64, 32, 16], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 5s 275ms/step - loss: 2.8060 - accuracy: 0.2496 - val_loss: 2.9227 - val_accuracy: 0.1868
Epoch 2/100
15/15 [==============================] - 4s 252ms/step - loss: 2.1718 - accuracy: 0.5336 - val_loss: 2.8726 - val_accuracy: 0.1612
Epoch 3/100
15/15 [==============================] - 4s 252ms/step - loss: 1.7427 - accuracy: 0.6366 - val_loss: 2.8778 - val_accuracy: 0.1172
Epoch 4/100
15/15 [==============================] - 4s 252ms/step - loss: 1.4820 - accuracy: 0.7262 - val_loss: 2.9222 - val_accuracy: 0.1136
Epoch 5/100
15/15 [==============================] - 4s 252ms/step - loss: 1.3012 - accuracy: 0.7846 - val_loss: 2.9889 - val_accuracy: 0.1978
Epoch 6/100
15/15 [=====================

In [129]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.5, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,10], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.5, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 10], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 3s 158ms/step - loss: 3.3084 - accuracy: 0.2464 - val_loss: 3.5479 - val_accuracy: 0.2857
Epoch 2/100
15/15 [==============================] - 2s 144ms/step - loss: 2.7550 - accuracy: 0.4430 - val_loss: 3.5124 - val_accuracy: 0.1209
Epoch 3/100
15/15 [==============================] - 2s 143ms/step - loss: 2.4254 - accuracy: 0.5503 - val_loss: 3.4938 - val_accuracy: 0.1648
Epoch 4/100
15/15 [==============================] - 2s 144ms/step - loss: 2.2061 - accuracy: 0.6079 - val_loss: 3.4788 - val_accuracy: 0.1722
Epoch 5/100
15/15 [==============================] - 2s 144ms/step - loss: 2.0614 - accuracy: 0.6307 - val_loss: 3.4935 - val_accuracy: 0.2161
Epoch 6/100
15/15 [=======================

In [138]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=50, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=50, dropout=0.66, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 3s 156ms/step - loss: 3.1267 - accuracy: 0.4048 - val_loss: 3.4661 - val_accuracy: 0.4872
Epoch 2/100
15/15 [==============================] - 2s 143ms/step - loss: 2.1509 - accuracy: 0.7394 - val_loss: 3.4049 - val_accuracy: 0.2637
Epoch 3/100
15/15 [==============================] - 2s 144ms/step - loss: 1.8856 - accuracy: 0.7956 - val_loss: 3.4737 - val_accuracy: 0.2015
Epoch 4/100
15/15 [==============================] - 2s 150ms/step - loss: 1.7741 - accuracy: 0.8311 - val_loss: 3.5582 - val_accuracy: 0.2088
Epoch 5/100
15/15 [==============================] - 2s 144ms/step - loss: 1.7518 - accuracy: 0.8316 - val_loss: 3.6521 - val_accuracy: 0.2088
Epoch 6/100
15/15 [=====================

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=50, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.5, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=50, dropout=0.5, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 3s 159ms/step - loss: 2.8992 - accuracy: 0.4857 - val_loss: 3.4703 - val_accuracy: 0.3846
Epoch 2/100
15/15 [==============================] - 2s 146ms/step - loss: 1.9935 - accuracy: 0.7897 - val_loss: 3.4438 - val_accuracy: 0.2051
Epoch 3/100
15/15 [==============================] - 2s 146ms/step - loss: 1.7560 - accuracy: 0.8453 - val_loss: 3.5366 - val_accuracy: 0.1905
Epoch 4/100
15/15 [==============================] - 2s 146ms/step - loss: 1.6682 - accuracy: 0.8661 - val_loss: 3.6518 - val_accuracy: 0.1538
Epoch 5/100
15/15 [==============================] - 2s 146ms/step - loss: 1.6350 - accuracy: 0.8741 - val_loss: 3.7777 - val_accuracy: 0.1209
Epoch 6/100
15/15 [======================

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=50, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.5, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

In [17]:
#!/usr/bin/env python3
#
# Team members' IDs:
# 182a3da8-8a9e-11ec-986f-f39926f24a9c  (Jan Zubáč)
# 7797f596-9326-11ec-986f-f39926f24a9c
# 449dba85-9adb-11ec-986f-f39926f24a9c
#
#
import argparse
import datetime
import os
import re
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import numpy as np
import tensorflow as tf

from modelnet import ModelNet

# TODO: Define reasonable defaults and optionally more parameters.
# Also, you can set the number of the threads 0 to use all your CPU cores.
# parser = argparse.ArgumentParser()
# parser.add_argument("--batch_size", default=64, type=int, help="Batch size.")
# parser.add_argument("--epochs", default=1, type=int, help="Number of epochs.")
# parser.add_argument("--modelnet", default=20, type=int, help="ModelNet dimension.")
# parser.add_argument("--seed", default=42, type=int, help="Random seed.")
# parser.add_argument("--threads", default=1, type=int, help="Maximum number of threads to use.")

# parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
# parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
# parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
# parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
# parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
# parser.add_argument("--dropout", default=0.3, type=float, help="Dropout regularization.")
# parser.add_argument("--hidden_layers", default=[100,100], nargs="*", type=int, help="Hidden layer sizes.")
# parser.add_argument("--l2", default=0.0, type=float, help="L2 regularization.")

# parser.add_argument("--filter_sizes", default=[128, 64, 32, 16, 2] ,  nargs="*", type=int, help="Filters for deconv-")


def main(args: argparse.Namespace) -> None:
    # Fix random seeds and threads
    tf.keras.utils.set_random_seed(args.seed)
    tf.config.threading.set_inter_op_parallelism_threads(args.threads)
    tf.config.threading.set_intra_op_parallelism_threads(args.threads)

    # Create logdir name
    args.logdir = os.path.join("logs", "{}-{}-{}".format(
        os.path.basename(globals().get("__file__", "notebook")),
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    ))

    # Load the data
    modelnet = ModelNet(args.modelnet)

    inputs = tf.keras.layers.Input(shape=[modelnet.D, modelnet.H, modelnet.W, modelnet.C], dtype=tf.float32)

    ## conv3D layers
    out = inputs
    for i, filter_size in enumerate(args.filter_sizes):
            conv3D =  tf.keras.layers.Conv3D(filters=filter_size,
                                        kernel_size=3,
                                        strides=(1, 1, 1),
                                        padding='same',
                                        data_format="channels_last",
                                        kernel_regularizer=tf.keras.regularizers.L2(l2=args.l2))
            
            # print("out1", out.shape)                                
            out = tf.concat( [conv3D(out), out], axis=-1)
            # print("out2", out.shape)                                


            # if i != len(args.filter_sizes)-1:
            out = tf.keras.layers.MaxPool3D(pool_size=(3, 3, 3)) (out)
            out = tf.keras.layers.BatchNormalization()(out)
            # out = tf.keras.layers.BatchNormalization()(out)
            # out = tf.keras.layers.Dropout(args.dropout)(out)
            # print("out", out.shape)                                

    out = tf.keras.layers.Flatten()(out)

    ## dense layers
    hidden = out
    for hidden_layer_size in args.hidden_layers:
        hidden =  tf.keras.layers.Dense(hidden_layer_size, activation=tf.nn.relu) (hidden)
        hidden = tf.keras.layers.Dropout(args.dropout) (hidden)

    outputs = tf.keras.layers.Dense(len(modelnet.LABELS), activation=tf.nn.softmax)(hidden)

    
    # choosing lr
    learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(  args.learning_rate, args.decay_steps, 
                                                                    end_learning_rate=args.learning_rate_final, power=1.0,
                                                                    cycle=False, name='linear_lr_decay')

    # choosing optimizer
    if args.optimizer=='SGD':
        if args.momentum != None:
            momentum = args.momentum
        else:
            momentum = 0.0
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=False, name='SGD')

    if args.optimizer=='Adam':
        optimizer =  tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')



    # TODO: Create the model and train it
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=optimizer,
        loss=tf.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.metrics.SparseCategoricalAccuracy("accuracy")],
        )
    

    # # train, dev = modelnet.train.data, modelnet.dev.data


    #     # # Create input data pipeline.
    def create_dataset(name):

        dataset = getattr(modelnet, name).dataset
        dataset = dataset.map(lambda example: (example["voxels"], example["labels"]))
        dataset = dataset.shuffle(len(dataset), seed=args.seed) if name == "train" else dataset
        dataset = dataset.batch(args.batch_size)
        return dataset

    train, dev, test = create_dataset("train"), create_dataset("dev"), create_dataset("test")

    tb_callback = tf.keras.callbacks.TensorBoard(args.logdir, update_freq=100, profile_batch=0)
    logs = model.fit(train, epochs=args.epochs, validation_data=dev)

    # Generate test set annotations, but in `args.logdir` to allow parallel execution.
    os.makedirs(args.logdir, exist_ok=True)
    with open(os.path.join(args.logdir, "3d_recognition.txt"), "w", encoding="utf-8") as predictions_file:
        # TODO: Predict the probabilities on the test set
        test_probabilities = model.predict(test)

        for probs in test_probabilities:
            print(np.argmax(probs), file=predictions_file)


# if __name__ == "__main__":
#     args = parser.parse_args([] if "__file__" not in globals() else None)
#     main(args)


In [14]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 3s 139ms/step - loss: 3.4541 - accuracy: 0.4018 - val_loss: 3.6584 - val_accuracy: 0.3516
Epoch 2/100
15/15 [==============================] - 2s 126ms/step - loss: 2.3183 - accuracy: 0.7539 - val_loss: 3.6082 - val_accuracy: 0.2674
Epoch 3/100
15/15 [==============================] - 2s 125ms/step - loss: 1.9949 - accuracy: 0.8281 - val_loss: 3.8754 - val_accuracy: 0.1758
Epoch 4/100
15/15 [==============================] - 2s 126ms/step - loss: 1.8192 - accuracy: 0.8666 - val_loss: 4.3497 - val_accuracy: 0.1136
Epoch 5/100
15/15 [==============================] - 2s 126ms/step - loss: 1.6922 - accuracy: 0.8932 - val_loss: 4.8291 - val_accuracy: 0.1099
Epoch 6/100
15/15 [====================

In [18]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=256, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.66, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=256, decay_steps=500, dropout=0.66, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
15/15 [==============================] - 3s 137ms/step - loss: 3.4451 - accuracy: 0.4072 - val_loss: 3.6500 - val_accuracy: 0.4652
Epoch 2/100
15/15 [==============================] - 2s 125ms/step - loss: 2.3287 - accuracy: 0.7453 - val_loss: 3.5668 - val_accuracy: 0.3407
Epoch 3/100
15/15 [==============================] - 2s 125ms/step - loss: 2.0001 - accuracy: 0.8268 - val_loss: 3.8232 - val_accuracy: 0.2015
Epoch 4/100
15/15 [==============================] - 2s 125ms/step - loss: 1.8121 - accuracy: 0.8739 - val_loss: 4.2760 - val_accuracy: 0.1245
Epoch 5/100
15/15 [==============================] - 2s 125ms/step - loss: 1.6836 - accuracy: 0.8973 - val_loss: 4.7442 - val_accuracy: 0.1099
Epoch 6/100
15/15 [====================

In [19]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=512, type=int, help="Batch size.")
parser.add_argument("--epochs", default=100, type=int, help="Number of epochs.")
parser.add_argument("--modelnet", default=32, type=int, help="ModelNet dimension.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=0, type=int, help="Maximum number of threads to use.")

parser.add_argument("--decay_steps", default=500, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.01, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.001, type=float, help="Final learning rate.")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
parser.add_argument("--dropout", default=0.6, type=float, help="Dropout regularization.")
parser.add_argument("--hidden_layers", default=[1024,1024], nargs="*", type=int, help="Hidden layer sizes.")
parser.add_argument("--l2", default=0.01, type=float, help="L2 regularization.")
parser.add_argument("--filter_sizes", default=[32,64,128] ,  nargs="*", type=int, help="Filters for deconv-")


args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=512, decay_steps=500, dropout=0.6, epochs=100, filter_sizes=[32, 64, 128], hidden_layers=[1024, 1024], l2=0.01, learning_rate=0.01, learning_rate_final=0.001, modelnet=32, momentum=0.85, optimizer='SGD', seed=42, threads=0)
Epoch 1/100
8/8 [==============================] - 4s 270ms/step - loss: 3.7588 - accuracy: 0.2848 - val_loss: 3.7864 - val_accuracy: 0.2784
Epoch 2/100
8/8 [==============================] - 2s 227ms/step - loss: 2.7019 - accuracy: 0.6555 - val_loss: 3.6532 - val_accuracy: 0.4762
Epoch 3/100
8/8 [==============================] - 2s 227ms/step - loss: 2.2572 - accuracy: 0.7684 - val_loss: 3.6167 - val_accuracy: 0.3004
Epoch 4/100
8/8 [==============================] - 2s 226ms/step - loss: 2.0327 - accuracy: 0.8254 - val_loss: 3.6799 - val_accuracy: 0.3004
Epoch 5/100
8/8 [==============================] - 2s 226ms/step - loss: 1.8988 - accuracy: 0.8596 - val_loss: 3.7978 - val_accuracy: 0.2747
Epoch 6/100
8/8 [==============================] -